In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download
from sklearn.datasets import fetch_20newsgroups
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from spacy import load
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

download('punkt')
download('stopwords')
download('omw-1.4')
download('wordnet')
nltk.download('wordnet2022')
#nlp = load('en_core_web_sm')


cp: /usr/share/nltk_data/corpora/wordnet2022: No such file or directory


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/siminjawed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siminjawed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/siminjawed/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/siminjawed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     /Users/siminjawed/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!


# Import Dataset

In [10]:
dataset = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)

In [11]:
df = pd.DataFrame()
df['text'] = dataset.data
df['encoded_labels'] = dataset.target

In [14]:
label=[]
for i in df['encoded_labels']:
    label.append(dataset.target_names[i])
df['label']=label

In [15]:
df.head()

,text,encoded_labels,label
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space


In [16]:
df.drop(['encoded_labels'],axis=1,inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11314 entries, 0 to 11313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11314 non-null  object
 1   label   11314 non-null  object
dtypes: object(2)
memory usage: 176.9+ KB


In [18]:
df['label'].value_counts()

label
rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: count, dtype: int64

## Combining Subcategories for each Label

In [19]:
df['label'].replace({'talk.politics.misc':'politics','talk.politics.guns':'politics','talk.politics.mideast':'politics'},inplace=True)                    
df['label'].replace({'rec.sport.hockey':'sport','rec.sport.baseball':'sport'},inplace=True)
df['label'].replace({'soc.religion.christian':'religion','talk.religion.misc':'religion', 'alt.atheism':'religion'},inplace=True)
df['label'].replace({'comp.windows.x':'computer','comp.sys.ibm.pc.hardware':'computer','comp.os.ms-windows.misc':'computer','comp.graphics':'computer','comp.sys.mac.hardware':'computer'},inplace=True)  
df['label'].replace({'misc.forsale':'sales'},inplace=True)
df['label'].replace({'rec.autos':'automobile','rec.motorcycles':'automobile'},inplace=True)
df['label'].replace({'sci.crypt':'science','sci.electronics':'science','sci.space':'science'},inplace=True)
df['label'].replace({'sci.med':'medicine'},inplace=True)

/var/folders/d8/xrxwydmx6sjf_0_vj9t4s1t80000gn/T/ipykernel_72957/674604520.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].replace({'talk.politics.misc':'politics','talk.politics.guns':'politics','talk.politics.mideast':'politics'},inplace=True)


In [20]:
df.label.value_counts()

label
computer      2936
science       1779
politics      1575
religion      1456
sport         1197
automobile    1192
medicine       594
sales          585
Name: count, dtype: int64

In [21]:
for i in range(0,10):
    print(df['text'][i])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. T

# Data Cleaning

In [22]:
df['Number_of_words'] = df['text'].apply(lambda x:len(str(x).split()))
df['Number_of_words'].describe()

count    11314.000000
mean       185.827382
std        523.971647
min          0.000000
25%         40.000000
50%         83.000000
75%        167.000000
max      11765.000000
Name: Number_of_words, dtype: float64

In [23]:
no_text = df[df['Number_of_words']==0]
print(len(no_text))

df.drop(no_text.index,inplace=True)

300


In [24]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[\d\W_]+', ' ', text)
    return text

df['text'] = df['text'].apply(lambda x: clean_text(x))

df['text'].head()

0    i was wondering if anyone out there could enli...
1    a fair number of brave souls who upgraded thei...
2    well folks my mac plus finally gave up the gho...
3     do you have weitek s address phone number i d...
4    from article c owcb n p world std com by tomba...
Name: text, dtype: object

In [25]:
df['text'] = df['text'].apply(nltk.word_tokenize)

df['text'].head()

0    [i, was, wondering, if, anyone, out, there, co...
1    [a, fair, number, of, brave, souls, who, upgra...
2    [well, folks, my, mac, plus, finally, gave, up...
3    [do, you, have, weitek, s, address, phone, num...
4    [from, article, c, owcb, n, p, world, std, com...
Name: text, dtype: object

In [26]:
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words]
df['text'] = df['text'].apply(remove_stopwords)

In [27]:
df['text']

0        [wondering, anyone, could, enlighten, car, saw...
1        [fair, number, brave, souls, upgraded, si, clo...
2        [well, folks, mac, plus, finally, gave, ghost,...
3        [weitek, address, phone, number, like, get, in...
4        [article, c, owcb, n, p, world, std, com, tomb...
                               ...                        
11309    [dn, nyeda, cnsvax, uwec, edu, david, nye, dn,...
11310    [old, mac, k, mac, plus, problem, screens, bla...
11311    [installed, dx, cpu, clone, motherboard, tried...
11312    [require, hyper, sphere, space, points, specif...
11313    [stolen, pasadena, pm, blue, white, honda, cbr...
Name: text, Length: 11014, dtype: object

In [28]:
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]
df['text'] = df['text'].apply(lemmatize_tokens)

df['text']

0        [wondering, anyone, could, enlighten, car, saw...
1        [fair, number, brave, soul, upgraded, si, cloc...
2        [well, folk, mac, plus, finally, gave, ghost, ...
3        [weitek, address, phone, number, like, get, in...
4        [article, c, owcb, n, p, world, std, com, tomb...
                               ...                        
11309    [dn, nyeda, cnsvax, uwec, edu, david, nye, dn,...
11310    [old, mac, k, mac, plus, problem, screen, blan...
11311    [installed, dx, cpu, clone, motherboard, tried...
11312    [require, hyper, sphere, space, point, specifi...
11313    [stolen, pasadena, pm, blue, white, honda, cbr...
Name: text, Length: 11014, dtype: object

In [29]:
def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

df['text'] = df['text'].apply(lambda x : combine_text(x))
df.head()

,text,label,Number_of_words
0,wondering anyone could enlighten car saw day d...,automobile,91
1,fair number brave soul upgraded si clock oscil...,computer,90
2,well folk mac plus finally gave ghost weekend ...,computer,307
3,weitek address phone number like get informati...,computer,15
4,article c owcb n p world std com tombaker worl...,science,72


In [30]:
df['Number_of_words'] = df['text'].apply(lambda x:len(str(x).split()))

In [31]:
df['Number_of_words'].describe()

count    11014.000000
mean       119.811694
std        559.505846
min          0.000000
25%         23.000000
50%         45.000000
75%         88.000000
max      14724.000000
Name: Number_of_words, dtype: float64

In [32]:
print(len(df[df['Number_of_words']==0]))

df.drop(df[df['Number_of_words']==0].index,inplace=True)

14


# Data Preprocessing

In [33]:
label_encoder = preprocessing.LabelEncoder()
df['target']= label_encoder.fit_transform(df['label'])

In [34]:
df

,text,label,Number_of_words,target
0,wondering anyone could enlighten car saw day d...,automobile,44,0
1,fair number brave soul upgraded si clock oscil...,computer,54,1
2,well folk mac plus finally gave ghost weekend ...,computer,147,1
3,weitek address phone number like get informati...,computer,8,1
4,article c owcb n p world std com tombaker worl...,science,46,6
...,...,...,...,...
11309,dn nyeda cnsvax uwec edu david nye dn neurolog...,medicine,165,2
11310,old mac k mac plus problem screen blank someti...,computer,58,1
11311,installed dx cpu clone motherboard tried mount...,computer,50,1
11312,require hyper sphere space point specifies sph...,computer,25,1


In [35]:
X = df['text']
y = df['target']

### Encoding Text using Universal Sentence Encoder

In [38]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") 

In [39]:
def embed_sentences(text):
    return embed([text])[0]

In [40]:
X = X.apply(embed_sentences)

In [42]:
def convert_to_numpy(tensor):
    numpy_array = tensor.numpy()
    return numpy_array

In [43]:
X = X.apply(convert_to_numpy)

In [45]:
X = np.array(X.tolist())

### Converting Imbalanced Dataset into Balanced using SMOTE Technique

In [47]:
smote = SMOTE(random_state = 42)
X_smote, Y_smote = smote.fit_resample(X,y)

In [48]:
Y_smote.value_counts()

target
0    2860
1    2860
6    2860
3    2860
2    2860
4    2860
5    2860
7    2860
Name: count, dtype: int64

# Train Test Split

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, Y_smote, test_size = 0.20, random_state = 42)
X_train.shape, X_test.shape,y_train.shape, y_test.shape

((18304, 512), (4576, 512), (18304,), (4576,))

# Creating Neural Network of Bidirectional LSTM for Training Model

In [51]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(Bidirectional(LSTM(512)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/30
572/572 [==============================] - 7s 10ms/step - loss: 0.5922 - accuracy: 0.8177 - val_loss: 0.3978 - val_accuracy: 0.8625
Epoch 2/30
572/572 [==============================] - 6s 10ms/step - loss: 0.3665 - accuracy: 0.8805 - val_loss: 0.3573 - val_accuracy: 0.8853
Epoch 3/30
572/572 [==============================] - 6s 11ms/step - loss: 0.3282 - accuracy: 0.8928 - val_loss: 0.3479 - val_accuracy: 0.8794
Epoch 4/30
572/572 [==============================] - 6s 11ms/step - loss: 0.2963 - accuracy: 0.9025 - val_loss: 0.3358 - val_accuracy: 0.8877
Epoch 5/30
572/572 [==============================] - 5s 9ms/step - loss: 0.2671 - accuracy: 0.9111 - val_loss: 0.3209 - val_accuracy: 0.8890
Epoch 6/30
572/572 [==============================] - 6s 11ms/step - loss: 0.2431 - accuracy: 0.9183 - val_loss: 0.3082 - val_accuracy: 0.8964
Epoch 7/30
572/572 [==============================] - 6s 11ms/step - loss: 0.2130 - accuracy: 0.9291 - val_loss: 0.2921 - val_accuracy: 0.9047


# Visualization of Accuracy, Loss and Confusion Matrix

In [52]:
# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

143/143 [==============================] - 2s 9ms/step


In [53]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

y_test_classes = np.argmax(tf.keras.utils.to_categorical(y_test, num_classes=13), axis=1)

conf_mat = confusion_matrix(y_test_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='magma_r', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Saving Model and Labels in H5 and joblib format respectively

In [54]:
model.save('model.h5')

import joblib
joblib.dump(label_encoder, 'label_encoder.joblib')

/Users/siminjawed/personalProjects/MusicLyricsGen/.venv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['label_encoder1.joblib']